In [1]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interact_manual, widgets,fixed
import functools

In [2]:
def compute_alarm_proba(entropy_target=0.95,test_size=range(1024),Th_low=range(1024),Th_high=range(1024)):
    v_x=np.arange(test_size+1,dtype=np.float64)
    nb_p=100
    v_p=np.arange(nb_p,dtype=np.float64)/(nb_p-1)
    v_proba_false_alarms=np.zeros((nb_p,),dtype=np.float64)
    v_h=np.zeros((nb_p,),dtype=np.float64)
    for p_i in range(nb_p):
        rv = binom(test_size, v_p[p_i])
        v_y=rv.pmf(v_x)
        proba_alarm=v_y[:Th_low].sum()+v_y[Th_high:].sum()
        v_proba_false_alarms[p_i]=proba_alarm
        if abs(0.5-v_p[p_i])!=0.5:
            v_h[p_i]=((v_p[p_i])*np.log2(v_p[p_i])+(1-v_p[p_i])*np.log2(1-v_p[p_i]))*-1
    v_tmp=np.where(v_h>entropy_target)[0]
    p0=v_p[v_tmp[0]]
    p1=v_p[v_tmp[-1]]
    fig, ax1 = plt.subplots(figsize=[7,5])
    ax1.set_xlabel('Probabilite de generer un 1')
    ax1.set_ylabel('Probabilite d avoir une alarme', color='tab:red')
    ax1.plot(v_p,v_proba_false_alarms, color='tab:red')
    ax2 = ax1.twinx()
    ax2.plot(v_p,v_h, color='tab:green')
    ax2.plot([p0,p0],[0,1], color='tab:blue')
    ax2.plot([p1,p1],[0,1], color='tab:blue')
    ax2.plot([p0,1],[v_h[v_tmp[0]],v_h[v_tmp[1]]], color='tab:blue')
    ax2.text(p1+0.01, 0.01+v_h[v_tmp[0]], 'Objectif d entropie=%.2f'%v_h[v_tmp[0]], fontsize=12,color='tab:blue')
    ax2.set_ylabel('-p.log2(p)-(1-p).log2(1-p)', color='tab:green')

In [4]:
im = interact(compute_alarm_proba)
im.widget.children[0].max=1-1E-10
im.widget.children[0].min=1E-10
im.widget.children[0].step=1E-2
im.widget.children[0].description='Objectif d entropie:'
im.widget.children[0].style={'description_width': '50%'}
im.widget.children[0].layout=widgets.Layout(width='100%')

im.widget.children[1].value=512
im.widget.children[1].description='Latence'
im.widget.children[1].style={'description_width': '50%'}
im.widget.children[1].layout=widgets.Layout(width='100%')

im.widget.children[2].value=12
im.widget.children[2].max=4096
im.widget.children[2].min=0
im.widget.children[2].step=1
im.widget.children[2].description='Seuil Bas'
im.widget.children[2].style={'description_width': '50%'}
im.widget.children[2].layout=widgets.Layout(width='100%')

im.widget.children[3].value=500
im.widget.children[3].max=4096
im.widget.children[3].min=0
im.widget.children[3].step=1
im.widget.children[3].description='Seuil Haut'
im.widget.children[3].style={'description_width': '50%'}
im.widget.children[3].layout=widgets.Layout(width='100%')

interactive(children=(FloatSlider(value=0.95, description='entropy_target', max=2.8499999999999996, min=-0.95)…